In [10]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

import pandas as pd

pyro.set_rng_seed(101)

def scale(guess):
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    measurement = pyro.sample("measurement", dist.Normal(weight, 0.75))
    return guess, weight.item(), measurement

df = pd.DataFrame([pyro.condition(scale, data={"measurement": 9.5})(8.5) for _ in range(100)], columns=['guess', 'weight', 'measurement'])
df.mean()

guess          8.500000
weight         8.429486
measurement    9.500000
dtype: float64

In [41]:
def serial():
    X = pyro.sample('X', dist.Normal(10, 1))
    Z = pyro.sample('Z', dist.Normal(2 * X, 1))
    Y = pyro.sample('Y', dist.Normal(3 * Z, 1))
    
    X = X.item() if isinstance(X, torch.Tensor) else X
    Z = Z.item() if isinstance(Z, torch.Tensor) else Z
    Y = Y.item() if isinstance(Y, torch.Tensor) else Y
    
    return X, Z, Y

In [42]:
pd.DataFrame([serial() for _ in range(10000)]).mean()

0    10.007839
1    20.021110
2    60.071404
dtype: float64

In [43]:
pd.DataFrame([pyro.condition(serial, data={'X': 15})() for _ in range(10000)]).mean()

0    15.000000
1    30.006949
2    90.021622
dtype: float64

In [44]:
pd.DataFrame([pyro.condition(serial, data={'Z': 10})() for _ in range(10000)]).mean()

0    10.001514
1    10.000000
2    30.012987
dtype: float64

In [45]:
pd.DataFrame([pyro.condition(serial, data={'Y': 7})() for _ in range(10000)]).mean()

0     9.981798
1    19.950718
2     7.000000
dtype: float64

* http://pyro.ai/examples/intro_part_ii.html